이전(https://www.kaggle.com/code/jihoryu1/eda-using-ta-lib/script) 에 다뤘던 EDA에서 이어집니다.<br>
저번 노트북에서는 TA-Lib를 사용하여 분석을 진행하였습니다.<br>
불행히도 제출시 대회 정책때문에 노트북에서 pip을 이용하여 TA를 설치하는 것은 불가능합니다.<br>
이를 해결하는 방법은 1. TA-Lib 사용후 데이터파일 별도 저장후 추가 2. TA를 사용하지 않고 직접 계산 이렇게 2가지 입니다.<br>
이번에는 2번을 사용해 보겠습니다.<br>
저번에 있던 지표중 비교적 효과적이라 생각했던 MACD, BB지표들을 직접 구현하했습니다.<br>

MACD의 경우 12일 이동평균선 - 26일 이동평균선 값이며<br>
BB의 경우 20일 이동평균선 +- 표준편차 * 2 입니다.<br>
본 문서에서는 해당 값들을 어떤식으로 feature로 편입시킬지에 대한 간단한 예시가 있습니다.<br>
해당 예시는 이런 방법이 있다 정도로만 참고해 주십시오.<br>
이번 노트북은 주석으로 설명하는 편이 이해하기 편하기에 이 이후의 설명은 주석으로 진행합니다.

In [ ]:
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from pathlib import Path
import jpx_tokyo_market_prediction
import optuna
import copy
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from decimal import ROUND_HALF_UP, Decimal
mpl.style.use('seaborn')

valid_df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=True)
# supplemental_files -> 20211206이후의 데이터 모음

prices = valid_df



prices = prices.drop(["ExpectedDividend"],axis=1)

prices = prices.interpolate(method='linear',limit_direction='both') # nan들 앞뒤 값들 참고해서 보정
prices.head()

In [ ]:
prices.info()

In [ ]:
average = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
def get_avg(_id_):
    return average.loc[_id_]

def getadvance(x):
    ret = 0
    if x > 0:
        ret = 1
    return(ret)


def cat_col(data) : # category화 가능한 값들은 category  적용 (속도 메모리 측면 이득)
    data['SecuritiesCode'] = data['SecuritiesCode'].astype('category')
    data['SupervisionFlag'] = data['SupervisionFlag'].astype('category')
    data['advance'] = data['advance'].astype('category')
    data['AdjustmentFactor'] = data['AdjustmentFactor'].astype('category')
    data['macd'] = data['macd'].astype('category')
    data['BB_check'] = data['BB_check'].astype('category')
    return data

def cal_macd(data1, data2): # macd에 대한 pos, neg 조정
    ret = 0
    if (data1 > data2):
        ret = 1
    elif (data1 < data2):
        ret = -1
    else:
        ret = 0
        
    return ret

def cal_BB(up, low, close, ma20): # BB에 대한 pos, neg 조정
    ret = 0
    if not (ma20 == 0):
        if (up <= close):
            ret = 1
        elif (low >= close):
            ret = -1
        else:
            ret = 0
    
    return ret

In [ ]:
prices.columns

In [ ]:
SecuritiesCode_list = prices["SecuritiesCode"].copy()
SecuritiesCode_list = list(set(SecuritiesCode_list))
SecuritiesCode_list.sort()

In [ ]:
# feature 값 추가전 공간확보

prices["Avg"] = 0
prices['pClose'] = 0
prices['delta'] = 0
prices['advance'] = 0
prices['Date'] = pd.to_datetime(prices['Date'], format = "%Y-%m-%d")

# -----------------------------------------------------------------------

prices['MA12'] =  0
prices['MA26'] =  0
prices['MA20'] =  0
prices['macd'] = 0
prices['stddev'] = 0
prices['upper'] = 0
prices['lower'] = 0
prices['BB_check'] = 0

In [ ]:
for i in SecuritiesCode_list: # 각 SecuritiesCode 별로 MACD, BB 계산
    temp_price = prices[prices["SecuritiesCode"] == i].copy()
    temp_price['pClose'] = temp_price['Close'].shift(-1)
    temp_price['delta'] = temp_price['Close'] - temp_price['pClose']
    temp_price['advance'] = list(map(getadvance, temp_price['delta']))
    temp_price["Avg"] = temp_price["SecuritiesCode"].apply(get_avg)

    # -----------------------------------------------------------------------

    temp_price['MA12'] =  temp_price['Close'].rolling(window=12).mean().values
    temp_price['MA26'] =  temp_price['Close'].rolling(window=26).mean().values
    temp_price['MA20'] =  temp_price['Close'].rolling(window=20).mean().values
    temp_price['macd'] = list(map(cal_macd, temp_price['MA12'], temp_price['MA26']))
    temp_price['stddev'] = temp_price['Close'].rolling(window=20).std().values # 20일 이동표준편차
    temp_price['upper'] = temp_price['MA20'] + 2*temp_price['stddev'] # 상단밴드
    temp_price['lower'] = temp_price['MA20'] - 2*temp_price['stddev'] # 하단밴드
    temp_price['BB_check'] = list(map(cal_BB, temp_price['upper'], temp_price['lower'], temp_price['Close'], temp_price['MA20']))
    
    # -----------------------------------------------------------------------
    
    prices.loc[prices["SecuritiesCode"] == i] = temp_price # SecuritiesCode 별로 feature 교체
    
prices = cat_col(prices)

In [ ]:
# nan 값들 보정 평균이동선은 특징상 특정 일수 전까지는 전부 nan으로 출력

prices['MA12'].fillna(0, inplace=True)
prices['MA26'].fillna(0, inplace=True)
prices['MA20'].fillna(0, inplace=True)
prices['stddev'].fillna(0, inplace=True)
prices['upper'].fillna(0, inplace=True)
prices['lower'].fillna(0, inplace=True)

In [ ]:
# feature로 사용할 항목들 지정
# 적절한 값을 넣어야 성능향상에 도움됨, 필요없는 값들은 noise로 작용가능성 있음

features = ['Date', 'SecuritiesCode', 'Open', 'High', 'Low', 'Close',
       'Volume', 'AdjustmentFactor', 'SupervisionFlag', 'Avg',
       'pClose', 'delta', 'advance', 'MA12', 'MA26', 'MA20', 'macd', 'BB_check']

In [ ]:
prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)

X = prices[features]
y = prices['Target']
codes = X.SecuritiesCode.unique()

In [ ]:
# 파라미터 튜닝 파트

def objectives(trial):
    params = {
            'num_leaves': trial.suggest_int('num_leaves', 20, 500),
            'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
            'max_bin': trial.suggest_int('max_bin', 2, 3000),
            'learning_rate': trial.suggest_uniform('learning_rate', 0, 1),
            'max_depth': trial.suggest_int('max_depth', -1, 3500),
            'boosting': 'goss'
    }

    model = LGBMRegressor(**params)
    model.fit(X,y)
    score = model.score(X,y)
    return score

In [ ]:
studyLGBM = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=0))
studyLGBM.optimize(objectives, n_trials=2)

trial = studyLGBM.best_trial
params_best = dict(trial.params.items())
params_best['random_seed'] = 0
    
model = LGBMRegressor(**params_best)#

In [ ]:
print('study.best_params:', studyLGBM.best_trial.value)
print('Number of finished trials:', len(studyLGBM.trials))
print('Best trial:', studyLGBM.best_trial.params)
print('study.best_params:', studyLGBM.best_params)

In [ ]:
# 파라미터 중요도 그래프로 표시
optuna.visualization.plot_param_importances(studyLGBM)

In [ ]:
model.fit(X,y)
model.score(X,y)

In [ ]:
# 이 파트 반드시 아래의 파트와 분리할것 make_env()에러가 심각함

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [ ]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    ds=[prices, options, financials, trades, secondary_prices, sample_prediction]
    
    prices["Avg"] = 0
    prices['pClose'] = 0
    prices['delta'] = 0
    prices['advance'] = 0
    prices['Date'] = pd.to_datetime(prices['Date'], format = "%Y-%m-%d")

    # -----------------------------------------------------------------------

    prices['MA12'].fillna(0, inplace=True)
    prices['MA26'].fillna(0, inplace=True)
    prices['MA20'].fillna(0, inplace=True)
    prices['macd'] = 0
    prices['stddev'] = 0
    prices['upper'] = 0
    prices['lower'] = 0
    prices['BB_check'] = 0
    
    # -----------------------------------------------------------------------
    
    SecuritiesCode_list = prices["SecuritiesCode"].copy()
    SecuritiesCode_list = list(set(SecuritiesCode_list))
    SecuritiesCode_list.sort()
    
    for i in SecuritiesCode_list:
        temp_price = prices[prices["SecuritiesCode"] == i].copy()
        temp_price2 = sample_prediction[sample_prediction["SecuritiesCode"] == i].copy()
        temp_price["Avg"] = temp_price2["SecuritiesCode"].apply(get_avg)
        temp_price['pClose'] = temp_price['Close'].shift(-1)
        temp_price['delta'] = temp_price['Close'] - temp_price['pClose']
        temp_price['advance'] = list(map(getadvance, temp_price['delta']))
    
        # -----------------------------------------------------------------------

        temp_price['MA12'] =  temp_price['Close'].rolling(window=12).mean().values
        temp_price['MA26'] =  temp_price['Close'].rolling(window=26).mean().values
        temp_price['MA20'] =  temp_price['Close'].rolling(window=20).mean().values
        temp_price['macd'] = list(map(cal_macd, temp_price['MA12'], temp_price['MA26']))
        temp_price['stddev'] = temp_price['Close'].rolling(window=20).std().values # 20일 이동표준편차
        temp_price['upper'] = temp_price['MA20'] + 2*temp_price['stddev'] # 상단밴드
        temp_price['lower'] = temp_price['MA20'] - 2*temp_price['stddev'] # 하단밴드
        temp_price['BB_check'] = list(map(cal_BB, temp_price['upper'], temp_price['lower'], temp_price['Close'], temp_price['MA20']))
    
        # -----------------------------------------------------------------------
        
        prices.loc[prices["SecuritiesCode"] == i] = temp_price
    
    prices = cat_col(prices)
    prices = prices.drop(labels='stddev',axis=1)
    prices = prices.drop(labels='upper',axis=1)
    prices = prices.drop(labels='lower',axis=1)
    
    prices['Date'] = prices['Date'].dt.strftime("%Y%m%d").astype(int)
    print('-------------------------------prices------------------------------')
    print(prices)
    print('------------------------------------------------------------------------------')    
    
    prices = prices.drop(['RowId','ExpectedDividend'],axis=1)

    sample_prediction["Prediction"] = model.predict(prices)
    print('-------sample_prediction--------')
    print(sample_prediction)
    sample_prediction = sample_prediction.sort_values(by = "Prediction", ascending=False)
    sample_prediction.Rank = np.arange(0,2000)
    sample_prediction = sample_prediction.sort_values(by = "SecuritiesCode", ascending=True)
    sample_prediction.drop(["Prediction"],axis=1)
    submission = sample_prediction[["Date","SecuritiesCode","Rank"]]
    print('-------------------------------submission------------------------------')
    print(submission)
    print('------------------------------------------------------------------------------')
    env.predict(submission)

이상으로 EDA와 Feature Engineering에 대한 예시와 설명을 마치겠습니다.